In [26]:
import numpy as np
import pandas as pd
from scipy.stats import norm,lognorm
from math import log, sqrt, pi, exp
import time
import plotly.express as px 
import plotly.graph_objects as go




r = 0.01
S = 30
K = 40
T = 240/365
vol = 0.30

def blackScholes(r,S,K,T,vol,type = "C"):


    d1 = (np.log(S/K) + (r+vol**2/2)*T)/(vol*np.sqrt(T))
    d2 = d1 - vol*np.sqrt(T)

    try:
        if type == "C":
            price = S*norm.cdf(d1,0,1) - K*np.exp(-r*T)*norm.cdf(d2,0,1)
        elif type == "P":
            price = K*np.exp(-r*T)*norm.cdf(-d2,0,1) - S*norm.cdf(-d1,0,1)
        else:
            None


        return price

    except:
        print("Type Needed")
        

    

In [27]:
blackScholes(r,S,K,T,vol,type = "C")

0.5132843798399405

In [28]:
# def d1np(S,K,T,r,vol):

#     return (np.log(S/K)+(r+vol**2/2.)*T)/(vol*np.sqrt(T))

# def d2np(S,K,T,r,vol):

#     return d1np(S,K,T,r,vol)-vol*np.sqrt(T)

In [29]:
def d1(S,K,T,r,vol):
    return(log(S/K)+(r+vol**2/2.)*T)/(vol*sqrt(T))
def d2(S,K,T,r,vol):
    return d1(S,K,T,r,vol)-vol*sqrt(T)

In [30]:

## Calculate Call-option price

def bs_call(S,K,T,r,vol):
    
    # S*N(d1) - K(e**-rt)N(d2)
    
    return S*norm.cdf(d1(S,K,T,r,vol))-K*exp(-r*T)*norm.cdf(d2(S,K,T,r,vol))

In [31]:

## Calculate Put-option price

def bs_put(S,K,T,r,vol):
 
    # K(e**-rt) - S + CallPrice    (Put-Call Parity)

    return K*exp(-r*T)-S+bs_call(S,K,T,r,vol)

In [32]:

# Greeks of Call option

def call_greeks(S,K,T,r,vol):

    delta = norm.cdf( d1(S,K,T,r,vol) )
    
    gamma = norm.pdf( d1(S,K,T,r,vol) ) / (S*vol*sqrt(T))

    vega  = ( S*norm.pdf( d1(S,K,T,r,vol) )*sqrt(T) ) /100

    theta = ( -(S*norm.pdf( d1(S,K,T,r,vol) )*vol) / (2*sqrt(T)) - r*K*exp(-r*T)*norm.cdf( d2(S,K,T,r,vol) ) ) /100

    rho   = ( K*T*exp(-r*T)*norm.cdf( d2(S,K,T,r,vol) ) ) /100

    return delta,gamma,vega,theta,rho



In [33]:
# Greeks of Put option

def put_greeks(S,K,T,r,vol):

    delta   = -norm.cdf( -d1(S,K,T,r,vol) )

    gamma   = norm.pdf( d1(S,K,T,r,vol) ) / (S*vol*sqrt(T))

    vega    = ( S*norm.pdf( d1(S,K,T,r,vol) ) * sqrt(T) ) /100

    theta   = ( -(S*norm.pdf( d1(S,K,T,r,vol) )*vol) / (2*sqrt(T)) + r*K*exp(-r*T)*norm.cdf( -d2(S,K,T,r,vol)) ) / 100

    rho     = ( -K*T*exp(-r*T)*norm.cdf( -d2(S,K,T,r,vol) )) /100

    return delta,gamma,vega,theta,rho



In [34]:
def vega(S, K, T, r, vol):

    d1 = ( log(S / K) + (r + vol ** 2 / 2) * T ) / vol * sqrt(T)
    
    return S * norm.pdf(d1) * sqrt(T)

In [35]:
def implied_volatility_grid(Price,S,K,T,r,option):

    vol = 0.001
    
    if option == 'C':
        while vol < 1:

            Price_implied = S*norm.cdf(d1(S,K,T,r,vol))-K*exp(-r*T)*norm.cdf(d2(S,K,T,r,vol))

            if Price-(Price_implied) < 0.001:

                return vol
            
            vol += 0.001

        return "It could not find the right volatility of the call option."
    
    else:

        while vol < 1:

            Price_implied = K*exp(-r*T)-S+bs_call(S,K,T,r,vol)

            if Price-(Price_implied) < 0.001:
                return vol
            
            vol += 0.001
            
        return "It could not find the right volatility of the put option."
    



In [36]:
def implied_volatility_call_NR(Price, S, K, T, r, tol=0.0001,max_iterations=500):

    # Use Newton-Raphson to iterate vol until bs_call matches price 

    # Starting Volatility

    vol = 0.4
    
    for i in range(max_iterations):

        diff = bs_call(S, K, T, r, vol) - Price

        if abs(diff) < tol:
            print(f'error: {diff}, {i} iterations')
            break

        vol = vol - diff / vega(S, K, T, r, vol)

    return vol

In [37]:
r = 0.01
S = 30
K = 40
T = 240/365
vol = 0.30
vega(S, K, T, r, vol)

7.702451712339081

In [38]:
r = 0.01
S = 30
K = 40
T = 240/365
vol = 0.30


call_price = bs_call(S,K,T,r,vol)
put_price = bs_put(S,K,T,r,vol)
# 
# print("BS call price :" + str(blackScholes(r,S,K,T,vol,type = "C")))

print("BS call price :" + str(bs_call(S,K,T,r,vol)))

# print("BS call price :" + str(bs_callnp(S,K,T,r,vol)))

# print("BS put price :" + str(blackScholes(r,S,K,T,vol,type = "P")))

print("BS put price :" + str(bs_put(S,K,T,r,vol)))

# print("BS put price :" + str(bs_putnp(S,K,T,r,vol)))


sigB = implied_volatility_grid(call_price,S,K,T,r,"C")

print("implied_volatility_grid: " + str(sigB))

sigNR = implied_volatility_call_NR(call_price, S, K, T, r, tol=0.0001,max_iterations=100)

print("implied_volatility_call_NR: " + str(sigNR))

BS call price :0.5132843798399405
BS put price :10.251133491653501
implied_volatility_grid: 0.3000000000000002
error: 4.054281338738974e-05, 7 iterations
implied_volatility_call_NR: 0.300007129287766


In [41]:
S = 500
vol = 0.3
r = 0.01

df = pd.DataFrame()

deltas =[]
gammas=[]
vegas=[]
thetas=[]
rhos=[]
Ks = []
Ts = []

lists = {}

T_range = range(1,240,2)
K_range = range(int(0.75*S),int(1.25*S),2)

# T_range = np.linspace(0.001,240,20)
# K_range = np.linspace(int(0.25*S),int(2.75*S),20)

T_list = [*T_range]
K_list = [*K_range]

for T in T_range:
    for K in K_range:

        delta,gamma,vega,theta,rho = call_greeks(S,K,T/365,r,vol)

        deltas.append(delta)
        gammas.append(gamma)
        vegas.append(vega)
        thetas.append(theta)
        rhos.append(rho)
        Ks.append(K)
        Ts.append(T)

df['K'] = Ks
df['T'] = Ts
df['Delta'] = deltas
# durr = np.reshape(deltas,(len(T_range),(len(K_range))))
df['Gamma'] = gammas
df['Vega'] = vegas
df['Theta'] = thetas
df['Rho'] = rhos

for col in df.columns:
    lists[col] = df[col].values


In [42]:
from scipy.interpolate import griddata

plots = ["Gamma","Vega","Theta"]

for plot in plots:
    
    x = np.array(Ks)
    y = np.array(Ts)
    z = lists[plot]

    xi = np.linspace(x.min(), x.max(), 100)
    yi = np.linspace(y.min(), y.max(), 100)

    X,Y = np.meshgrid(xi,yi)

    Z = griddata((x,y),z,(X,Y), method='cubic')

    fig = go.Figure(go.Surface(x=xi,y=yi,z=Z,colorscale ='Viridis'))
    fig.update_layout(
        autosize=False,
        scene = dict(
                    xaxis_title='Strike Price',
                    yaxis_title='Time to Expiry',
                    zaxis_title='Value'),
        # xaxis_title="Strike Price",
        # yaxis_title="Time to Expiry",
        width=800,
        height=800,
        title=plot,
        
    )

    fig.show()